In [1]:
import torch
import torch.utils.data as data
import torchvision.datasets.CIFAR100 as CIFAR100
import torchvision.transforms as transforms
import numpy as np

In [2]:
dataloader = datasets.CIFAR100
num_classes = 100

img_scale=(256, 256)
img_crop=224

transforms = transforms.Compose([
            transforms.Scale(img_scale),
            transforms.CenterCrop(img_crop),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])])

/home/tliu/.local/lib/python3.5/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [3]:
train_dataloader = dataloader(root='./cifar100_data', train=True, download=True, transform=transforms)
test_dataloader = dataloader(root='./cifar100_data', train=False, download=True, transform=transforms)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_dataloader)

50000

In [5]:
train_dataloader[0]

(tensor([[[ 2.2489,  2.2489,  2.2489,  ...,  0.9303,  0.8789,  0.8447],
          [ 2.2489,  2.2489,  2.2489,  ...,  0.9646,  0.9132,  0.8789],
          [ 2.2489,  2.2489,  2.2489,  ...,  0.9988,  0.9474,  0.9132],
          ...,
          [ 0.2453,  0.2453,  0.2282,  ..., -1.4843, -1.3987, -1.3130],
          [ 0.1939,  0.2111,  0.1939,  ..., -1.4843, -1.3987, -1.2959],
          [ 0.1597,  0.1768,  0.1597,  ..., -1.4843, -1.3815, -1.2788]],
 
         [[ 2.4286,  2.4286,  2.4286,  ...,  1.2206,  1.1856,  1.1681],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.2731,  1.2206,  1.2031],
          [ 2.4286,  2.4286,  2.4286,  ...,  1.3081,  1.2731,  1.2381],
          ...,
          [ 1.0455,  1.0455,  1.0455,  ..., -1.5630, -1.4405, -1.3179],
          [ 0.9930,  0.9930,  0.9930,  ..., -1.5455, -1.4055, -1.2654],
          [ 0.9405,  0.9405,  0.9580,  ..., -1.5280, -1.3704, -1.2304]],
 
         [[ 2.6400,  2.6400,  2.6400,  ...,  0.8971,  0.8274,  0.7576],
          [ 2.6400,  2.6400,

In [18]:
num_users = 50
num_examples = len(train_dataloader)

train_indices = np.random.permutation(num_examples)
train_indices = np.split(train_indices, num_users)

class CIFAR100Dataset(torch.utils.data.Dataset):
    def __init__(self, dataloader, indices):
        self.dataloader = dataloader
        self.indices = indices
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.dataloader[self.indices[idx]]
    
datasets = []
for i in range(num_users):
    datasets.append(CIFAR100Dataset(train_dataloader, train_indices[i]))
    
dataloaders = []
for i in range(num_users):
    dataloaders.append(torch.utils.data.DataLoader(datasets[i], batch_size=48, shuffle=True, num_workers=0))

In [22]:
class CIFAR100Dataset(Dataset):
    def __init__(self, dataloader, indices):
        self.dataloader = dataloader
        self.indices = indices
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.dataloader[self.indices[idx]]

def load_datasets_cifar(config, train=True, img_scale=(256, 256), img_crop=224, download=True, num_users=1):
    config = config['data_cifar']
    phase = 'train' if train else 'val'
    
    transforms = transforms.Compose([
                transforms.Scale(config['images']['scale']),
                transforms.CenterCrop(config['images']['crop']),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])])
    
    dataloader = dataloader(root=config[dir], train=train, download=True, transform=transforms)
    
    num_examples = len(dataloader)
    indices = np.random.permutation(num_examples)
    indices = np.split(indices, num_users)
    
    datasets = {}
    for i in range(num_users):
        datasets[i] = CIFAR100Dataset(train_dataloader, train_indices[i])
        
    dataloaders = {}
    for i in range(num_users):
        dataloaders[i] = DataLoader(datasets[i], batch_size=config[phase]['batch_size'], shuffle=True, num_workers=config['loader']['workers'])
        
    return dataloaders

In [24]:
train = False
'train' if train else 'val'

'val'

In [40]:
np.random.randint(4)

2